In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://ppa.lau

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GamingETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

# Filter By Votes


In [ ]:
df = df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df.show(10)

df = df.filter(df["total_votes"] >= 20)
df = df.filter(df["helpful_votes"]/df["total_votes"] >= 0.3)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|            2|          3|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [ ]:
df.count()

2060

# Divide by paid and unpaid

In [ ]:
purchase_df = df.filter(df['verified_purchase']== 'Y')
noPurchase_df = df.filter(df['verified_purchase']== 'N')

purchase_df.describe().show()
noPurchase_df.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|              1019|             1019|              1019|1019|             1019|
|   mean|3.0088321884200195| 43.9126594700687| 53.98331697742885|null|             null|
| stddev| 1.751485619845831|185.9758081176227|198.78697643553434|null|             null|
|    min|                 1|               10|               100|   N|                Y|
|    max|                 5|              993|                99|   N|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+

+-------+------------------+------------------+-----------------+----+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|
+-------+-----------

# Five Star Reviews among Verified purchase

In [ ]:
purchase_five_star = purchase_df[purchase_df['star_rating']== 5].count()
purchase_number = purchase_df.count()
percentage_five_star_purchase = float(purchase_five_star) / float(purchase_number)
print(purchase_number)
print(purchase_five_star)
print(percentage_five_star_purchase)

1019
360
0.35328753680078506


# Five Star Reviews among Non Verified purchase

In [ ]:
noPurchase_five_star = noPurchase_df[noPurchase_df['star_rating']== 5].count()
noPurchase_number = noPurchase_df.count()
percentage_five_star_noPurchase = float(noPurchase_five_star) / float(noPurchase_number)
print(noPurchase_number)
print(noPurchase_five_star)
print(percentage_five_star_noPurchase)

1041
323
0.3102785782901057


The data above shows that the total number of purchase and no purchase that provided a review is very close. So as the number of reviews with 5 stars for purchase vs no purchase. Which I consider strange because why would you give a 5 star review if you didn't purchase the product.